# Query cellxgene-census

In [ ]:
import lamindb as ln
import lnschema_bionty as lb
import cellxgene_census

In [ ]:
lb.settings.species = "human"
species = lb.settings.species.scientific_name

In [ ]:
ln.track()

## Query datasets by metadata

In [ ]:
modalities = ln.Modality.lookup()
features = ln.Feature.lookup(return_field=ln.Feature.name)
assays = lb.ExperimentalFactor.lookup(return_field=lb.ExperimentalFactor.name)
cell_types = lb.CellType.lookup(return_field=lb.CellType.name)
tissues = lb.Tissue.lookup(return_field=lb.Tissue.name)
diseases = lb.Disease.lookup(return_field=lb.Disease.name)
ulabels = ln.ULabel.lookup()
suspension_types = ulabels.is_suspension_type.children.all().lookup(
    return_field=ln.ULabel.name
)

In [ ]:
value_filter = (
    f'{features.tissue} == "{tissues.brain}" and {features.cell_type} in'
    f' ["{cell_types.microglial_cell}", "{cell_types.neuron}"] and'
    f' {features.suspension_type} == "{suspension_types.cell}" and {features.assay} =='
    f' "{assays.ln_10x_3_v3}"'
)

In [ ]:
value_filter

In [ ]:
with cellxgene_census.open_soma() as census:
    # Reads SOMADataFrame as a slice
    cell_metadata = census["census_data"][species].obs.read(value_filter=value_filter)

    # Concatenates results to pyarrow.Table
    cell_metadata = cell_metadata.concat()

    # Converts to pandas.DataFrame
    cell_metadata = cell_metadata.to_pandas()

In [ ]:
cell_metadata.shape

In [ ]:
cell_metadata.head()

In [ ]:
cell_metadata["dataset_id"].value_counts()

## Fetch the AnnData based on filters

In [ ]:
with cellxgene_census.open_soma() as census:
    adata = cellxgene_census.get_anndata(
        census=census,
        organism=species,
        obs_value_filter=value_filter,
        column_names={
            "obs": [
                features.assay,
                features.assay_ontology_term_id,
                features.cell_type,
                features.cell_type_ontology_term_id,
                features.tissue,
                features.tissue_ontology_term_id,
                features.disease,
                features.disease_ontology_term_id,
                features.suspension_type,
            ]
        },
    )

In [ ]:
adata.var = adata.var.set_index("feature_id")

In [ ]:
adata

In [ ]:
adata.var.head()

In [ ]:
adata.obs.head()

## Register the queried data

Register `AnnData`:

In [ ]:
file = ln.File.from_anndata(
    adata,
    description=(
        "microglial and neuron cell data from 10x 3' v3 in brain queried from Census"
    ),
    field=lb.Gene.ensembl_gene_id,
    modality=modalities.rna,
)

In [ ]:
file.save()

Link validated metadata:

In [ ]:
for col in adata.obs.columns:
    if not col.endswith("ontology_term_id"):
        feature = ln.Feature.filter(name=col).one()
        file.labels.add(adata.obs[col], feature)

In [ ]:
file.describe()

In [ ]:
# clean up test instance
!lamin delete --force test-census
!rm -r ./test-census